# Imports & Setup

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [2]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

spark

-rw-r--r-- 1 root root 247882 Jan 13 19:58 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [4]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'project_ir_os' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and "multistream" in b.name and ".parquet" in b.name:
        paths.append(full_path+b.name)
        print(full_path+b.name)
#     if b.name != 'graphframes.sh':
#         paths.append(full_path+b.name)

gs://project_ir_os/multistream10_preprocessed.parquet
gs://project_ir_os/multistream11_part2_preprocessed.parquet
gs://project_ir_os/multistream11_preprocessed.parquet
gs://project_ir_os/multistream12_part2_preprocessed.parquet
gs://project_ir_os/multistream12_preprocessed.parquet
gs://project_ir_os/multistream13_part2_preprocessed.parquet
gs://project_ir_os/multistream13_preprocessed.parquet
gs://project_ir_os/multistream14_part2_preprocessed.parquet
gs://project_ir_os/multistream14_preprocessed.parquet
gs://project_ir_os/multistream15_part2_preprocessed.parquet
gs://project_ir_os/multistream15_part3_preprocessed.parquet
gs://project_ir_os/multistream15_preprocessed.parquet
gs://project_ir_os/multistream16_part2_preprocessed.parquet
gs://project_ir_os/multistream16_part3_preprocessed.parquet
gs://project_ir_os/multistream16_preprocessed.parquet
gs://project_ir_os/multistream17_part2_preprocessed.parquet
gs://project_ir_os/multistream17_part3_preprocessed.parquet
gs://project_ir_os/mul

In [5]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

In [6]:
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

import math
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords) #to remark

RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.

    Parameters:
    -----------
    text: string , represting the text to tokenize.

    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if
                      token.group() not in all_stopwords]
    return list_of_tokens


def doc_length(text,id1):
    tokens = tokenize(text)
    d={}
    s=0
    for token in tokens:
        d[token]=d.get(token,0)+1
    for k,v in d.items():
        s+=v*v
    q=math.sqrt(s)
    return id1,q

lens=doc_text_pairs.map(lambda x:doc_length(x[0],x[1]))
dict_len=lens.collectAsMap()

In [8]:
import pickle
# Open a file to write the pickle to

index_src = "nf.pkl"
with open(index_src, "wb") as pickle_file:
    # Write the dictionary to the file
    pickle.dump(dict_len, pickle_file)
    
index_dst = f'gs://{bucket_name}/NF/{index_src}' # TODO::
!gsutil cp $index_src $index_dst


Copying file://nf.pkl [Content-Type=application/octet-stream]...
- [1 files][ 84.7 MiB/ 84.7 MiB]                                                
Operation completed over 1 objects/84.7 MiB.                                     
